In [1]:
import numpy as np
import pandas as pd
from numpy import linalg as LA
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import euclidean_distances
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1)
import csv
import os
import sklearn.metrics.pairwise as pdist
from sklearn.metrics.pairwise import euclidean_distances


In [2]:
# %load "packages_headers.py"
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from heapq import heapify, heappush, heappop

import numpy as np
from numpy import array
import scipy.sparse as sp
from sklearn.preprocessing import normalize
from scipy.sparse import diags
from scipy.sparse import csr_matrix
#import scipy.sparse as sp
import time
t_s = time.time()
#print("time passed:"+ str(time.time()-t_s))
import sklearn.metrics.pairwise as pdist
from sklearn.metrics.pairwise import euclidean_distances
import gc
#del(variable_name)
gc.collect()
import random

In [3]:
cwd = os.getcwd()
print(cwd)

/Users/chenzhenxu/CodeProjects/jupyter_notebook/Arcdata


In [4]:
datadr=cwd[:-4]+str("data/knn")
print(datadr)

/Users/chenzhenxu/CodeProjects/jupyter_notebook/Arcdata/knn


In [5]:
def getLabDic(lables):
    """input a list of lables
    return a dictionary mapping labels to 0,1,2,3...n-1
    """
    Labs=list(set(lables))
    n=len(Labs)
    labdic={}
    for i in range(n):
        labdic[Labs[i]]=i
    for k,v in labdic.items():
        print(str(k)+"--->"+str(v))
    return labdic

def getdatafromFile(filenm):
    """ this function uses the input filename
      assume each line: label +\t+ line
      return list of labels, list of lines
    """
    dat=[]
    Y=[]
    openf=open(filenm, 'r', encoding="latin1")
    for i, ln in enumerate(openf):
        ln2=ln.split('\t')
        Y.append(ln2[0])
        dat.append(ln2[1])
    openf.close()
    return (dat,Y)

In [6]:
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
test_r8fn=datadr+'/r8-test-all-terms.txt'
train_r8fn=datadr+'/r8-train-all-terms.txt'
test_20ngfn=datadr+'/20ng-test-all-terms.txt'
train_20ngfn=datadr+'/20ng-train-all-terms.txt'
train_ohfn=datadr+'/train_ohsumed_by_line.txt'
test_ohfn=datadr+'/test_ohsumed_by_line.txt'


In [8]:
test_r8fn

'/Users/chenzhenxu/CodeProjects/jupyter_notebook/Arcdata/knn/r8-test-all-terms.txt'

In [9]:
(traindat,trainY)=getdatafromFile(train_r8fn)
(testdat, testY)=getdatafromFile(test_r8fn)
r8Labdic=getLabDic(trainY)
train_lab=[r8Labdic[e] for e in trainY]
test_lab=[r8Labdic[e] for e in testY]

crude--->0
grain--->1
money-fx--->2
earn--->3
trade--->4
ship--->5
interest--->6
acq--->7


In [10]:
print(len(train_lab))
print(len(test_lab))
train_lab=np.asarray(train_lab)
test_lab=np.asarray(test_lab)

5485
2189


In [11]:
r8LabdicRev={}
for k,v in r8Labdic.items():
    r8LabdicRev[v]=k

In [12]:
docs=['I like Brandeis', 'machine learning', 'I work at Brandeis','machine learning is awesome']

In [13]:
vect = CountVectorizer(ngram_range=(1,1), stop_words=None)
N=len(docs)
print(f"there are { N } docs.")
tfx=vect.fit_transform(docs)
print(tfx.shape)
vocabs=vect.get_feature_names_out()
print(f"length of vocab={len(vocabs)}")
dfs=csr_matrix(((tfx>0)*1).sum(axis=0)).toarray()[0]

there are 4 docs.
(4, 8)
length of vocab=8


In [14]:
dfs

array([1, 1, 2, 1, 2, 1, 2, 1])

In [15]:
vocabs

array(['at', 'awesome', 'brandeis', 'is', 'learning', 'like', 'machine',
       'work'], dtype=object)

In [16]:
tfx.toarray()

array([[0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 0, 0, 0, 1],
       [0, 1, 0, 1, 1, 0, 1, 0]])

In [17]:
tfx2=normalize(tfx, norm='l1',axis=1)

In [18]:
tfx2.toarray()

array([[0.        , 0.        , 0.5       , 0.        , 0.        ,
        0.5       , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.5       ,
        0.        , 0.5       , 0.        ],
       [0.33333333, 0.        , 0.33333333, 0.        , 0.        ,
        0.        , 0.        , 0.33333333],
       [0.        , 0.25      , 0.        , 0.25      , 0.25      ,
        0.        , 0.25      , 0.        ]])

In [19]:
dfs=csr_matrix(((tfx>0)*1).sum(axis=0)).toarray()[0] # document frequency

In [20]:
dfs

array([1, 1, 2, 1, 2, 1, 2, 1])

In [21]:
def tfidf(traindocs, stopwordlist=None):
    vect = CountVectorizer(ngram_range=(1,1), stop_words=stopwordlist)
    N=len(traindocs)
    print(f"there are { N } docs.")
    tfx=vect.fit_transform(traindocs)
    print(tfx.shape)
    vocabs=vect.get_feature_names_out()
    print(f"length of vocab={len(vocabs)}")
    dfs=csr_matrix(((tfx>0)*1).sum(axis=0)).toarray()[0]
    print("shape of dfs:"+str(dfs.shape))
    print("len of dfs: "+str(len(dfs)))

    vocabs=vect.get_feature_names_out()
    print(f"length of vocab={len(vocabs)}")
    
    ### tfx1 matrix indicating existence of a word or not (1/0) for tfx0 purposes
    # tfx1=sp.csr_matrix((tfx>0)*1)
    ## l1 normalize to frequency for each row/doc
    #tfx=csr_matrix(tfx/tfx.sum(axis=1))

    tfx=normalize(tfx, norm='l1',axis=1)
    print("shape of tfx1: "+str(tfx.shape))
    print("computing tfidf ... ")
    idfs=[1+np.log(N/(1+d)) for d in dfs]
    idfs_diag=diags(idfs) 
    tfidf=tfx.dot(idfs_diag)
    return

In [22]:
tfidf(traindat)

there are 5485 docs.
(5485, 19956)
length of vocab=19956
shape of dfs:(19956,)
len of dfs: 19956
length of vocab=19956
shape of tfx1: (5485, 19956)
computing tfidf ... 


In [23]:
def evaluate(testdocs, testlabs): # testdocs is a list of docs
    predictionlabs=predict(testdocs)
    #prediction=np.asarray(prediction)
    count=0; ntest=len(predictionlabs)
    for i in range(ntest):
        if predictionlabs[i]==testlabs[i]:
            count=count+1
    test_error = 1 - count/ntest
    return test_error


def KNNpredict(neighbor_classes, C, cross_val=False):
    # Make sure all classes are considered
    labels = np.concatenate((neighbor_classes, list(range(C))))
    # Find class frequency among neighbors
    weights = np.unique(labels, return_counts=True)[1]
    # Find most popular class
    prediction = np.argmax(weights)
    nn=len(neighbor_classes)
    # If most popular class is ambiguous try with fewer neighbors; else return
    if sum(weights[prediction] == weights) > 1 and nn>2:
        return KNNpredict(neighbor_classes[:-2], C)
    else:
        return prediction



def knnClassify(y_train, dist, n_neighbors=7): 
    ## y_train: training docs labels; dist: testdocs vs traindocs pairwise distance matrix 
    # Number of all different classes
    #n_classes = len(np.unique(y_train))
    prediction = []
    ntestsample=dist.shape[0]
    for i in range(ntestsample):
        doc_to_train=dist[i,:]
        # Find indices of n_neighbors closest documents
        rank = np.argsort(doc_to_train)[:n_neighbors]
        # Make prediction based on most popular class among neighbors
        prediction.append(KNNpredict(y_train[rank], n_classes))
    return prediction
